In [1]:
#dependencies
import re
from bs4 import BeautifulSoup 
from splinter import Browser
import requests
import pandas as pd
from selenium import webdriver 
from selenium.common.exceptions import TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
import json
import time
from tqdm.notebook import tqdm 
import numpy as np

<h4>Start cells here if {state}_geo file has not been FIPS encoded

In [2]:
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_nd.csv") #- original geocoded file
fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1","Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1","Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
fips_check

,Unnamed: 0,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,546457,546457,546457,546457,546457,5206477102,4/13/20,875,PPP,EDGEWOOD OPCO LLC,...,999 Other,6233 Continuing Care Retirement Communities an...,6233.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,NaN
1,546458,546458,546458,546458,546458,2976507101,4/11/20,875,PPP,STRATA CORPORATION,...,999 Other,"2373 Highway, Street, and Bridge Construction",2373.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,250 to 499,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,NaN
2,546459,546459,546459,546459,546459,5041237003,4/4/20,875,PPP,MINN-DAK FARMERS COOPERATIVE,...,311 Food,3113 Sugar and Confectionery Product Manufactu...,3113.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,NaN
3,546460,546460,546460,546460,546460,4487847009,4/3/20,875,PPP,"MILLER INSULATION CO, INC.",...,999 Other,2362 Nonresidential Building Construction,2362.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,250 to 499,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,NaN
4,546461,546461,546461,546461,546461,2579588901,4/27/21,875,PPP,SISTERS OF MARY OF THE PRESENTATION LONG TERM ...,...,999 Other,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,549674,549674,549674,549674,549674,2849968506,2/22/21,875,PPS,G & R TRUCKING LLC,...,999 Other,4841 General Freight Trucking,4841.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"3913 106th Ave SE, Sanborn, ND",46.848916,-98.228272,NaN
3218,549675,549675,549675,549675,549675,3026678408,2/4/21,875,PPS,H.J. VENTURE CORP.,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1218 S Washington St, Grand Forks, ND",47.909497,-97.046115,NaN
3219,549676,549676,549676,549676,549676,3175408401,2/4/21,875,PPS,NORTHWEST WATER TRANSFER LLC,...,999 Other,2131 Support Activities for Mining,2131.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"5458 134th Ave NW, Williston, ND",48.205512,-103.580819,NaN
3220,549677,549677,549677,549677,549677,3860267409,5/8/20,875,PPP,OUTSIDER'S BAR AND GRILL LLC,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"905 3RD AVE SW, WATFORD CITY, ND",47.796467,-103.295171,NaN


In [3]:
columns = ['LoanNumber','full_add', 'Lat', 'Long']
fips_check = fips_check[columns]

In [4]:
fips_check_nan = fips_check[fips_check["Lat"].isnull()] # check to make sure no nulls in coords
fips_check_nan #3

,LoanNumber,full_add,Lat,Long


In [5]:
fips_check_notna = fips_check[fips_check["Lat"].notna()] # check to make sure no nulls in coords
fips_check_notna.head() #15866

,LoanNumber,full_add,Lat,Long
0,5206477102,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593
1,2976507101,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896
2,5041237003,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776
3,4487847009,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451
4,2579588901,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143


<h4> Start cells here if partial FIPS already available in {state}_fips_scraped file

In [6]:
fips_check.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/ND_fips_scraped.csv")


In [ ]:
fips_check_scraped = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/ND_fips_scraped.csv") 
fips_check_scraped.head() 

In [ ]:
columns = ['LoanNumber','full_add', 'Lat', 'Long', 'FIPS_z']
fips_check_scraped = fips_check_scraped[columns]

In [12]:
#pd.set_option('display.float_format', '{:.0f}'.format)

In [13]:
fips_check_scraped['Lat'].replace('None', np.nan, inplace=True)
fips_check_scraped['Long'].replace('None', np.nan, inplace=True)

In [14]:
#Latitude, Longitude, not included
fips_check_scraped_notna = fips_check_scraped[fips_check_scraped["Lat"].notna()] # check to make sure no nulls in coords
fips_check_scraped_notna #15866

,LoanNumber,full_add,Lat,Long,FIPS_z
0,4036167203,"11808 Grant St, Suite 100, Omaha, NE",41.280517,-96.098656,NaN
1,6046987205,"2225 Q ST, AURORA, NE",40.875357,-97.988962,NaN
2,7244777109,"15380 WEIR ST, OMAHA, NE",41.207606,-96.155808,NaN
3,7268117104,"11718 NICHOLAS ST STE 101, OMAHA, NE",41.269855,-96.097118,NaN
4,7983549008,"4980 S 118th St, Omaha, NE",41.207966,-96.098948,NaN
...,...,...,...,...,...
5799,6609317001,"17525 Arbor Street, OMAHA, NE",41.236225,-96.189536,NaN
5800,7946408605,"1045 N 115th St Ste 150, Omaha, NE",41.269145,-96.093016,NaN
5801,8534688606,"2110 S 67th St Ste 106, Omaha, NE",41.239763,-96.015807,NaN
5802,9113417203,"2605 South 84th St., OMAHA, NE",41.234633,-96.042815,NaN


In [15]:
#FIPS not included, of Lat, Long available
fips_check_scraped_null = fips_check_scraped_notna[fips_check_scraped_notna["FIPS_z"].isnull()] # check to make sure no nulls in coords
fips_check_scraped_null.count() 

LoanNumber    5803
full_add      5803
Lat           5803
Long          5803
FIPS_z           0
dtype: int64

<h4>FIPS Script

In [6]:
#if geocode file
lat = fips_check_notna["Lat"].astype(float).astype(str).to_list()
long = fips_check_notna["Long"].astype(float).astype(str).to_list()

#if FIPS_z file 
#lat = fips_check_scraped_null["Lat"].astype(float).astype(str).to_list()
#long = fips_check_scraped_null["Long"].astype(float).astype(str).to_list()

In [7]:
#before running script, check for any NaN within Lat, Long fields. 
def sleep():
    time.sleep(1)
    
service = {'service': Service(ChromeDriverManager().install())}
driver = Browser('chrome', **service, headless=True)    
url = 'https://geocoding.geo.census.gov/geocoder/geographies/coordinates?form'
driver.visit(url)

not_found = []
geo_id = []

i=0
for i in tqdm(range(len(lat))):
    #while i < len(lat):
        driver.find_by_id('geoCoordX').clear()
        driver.find_by_id('geoCoordY').clear()
        driver.find_by_id('geoCoordX').fill(long[i])
        driver.find_by_id('geoCoordY').fill(lat[i])
        driver.find_by_id('geoCoordsSubmit').click()
        #sleep()
        #html = driver.html
        soup = BeautifulSoup(driver.html, 'html.parser')


        results = soup.find_all('div', id='geoCoordsJsonOutput')
        for result in results:
            result_text = result.text

            result_digits = re.findall('\d+', result_text)
            startswith = [s for s in result_digits if s.startswith('38')] #ND
            fips_elements =[element for element in startswith if len(element)>=15]
            
            try:
                #print(f"{i},{lat[i]},{long[i]} is {fips_elements}")
                geo_id.append(fips_elements)
            except (KeyError, IndexError):
                #print(f"{i},{lat[i]},{long[i]} Not Found")
                notfound.append(i)
                geo_id.append(None)  
            except (TimeoutException):
                driver.navigate.refresh()
    
        #sleep()
        i=i+1



====== WebDriver manager ======
Current google-chrome version is 102.0.5005
Get LATEST chromedriver version for 102.0.5005 google-chrome
Driver [/Users/ciaraspencer/.wdm/drivers/chromedriver/mac64/102.0.5005.61/chromedriver] found in cache


  0%|          | 0/3222 [00:00<?, ?it/s]

https://iqss.github.io/dss-webscrape/filling-in-web-forms.html

In [10]:
geo_id_list =geo_id 

Length Check

In [11]:
print(len(geo_id_list))

3222


Matching FIPS to DataFrame

In [12]:
#for geocode file
fips_fill = fips_check_notna
#if FIPS available
#fips_fill = fips_check_scraped_null

In [13]:
fips_fill["FIPS_z"] = geo_id_list
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,5206477102,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,[380350101002024]
1,2976507101,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,[380350117011097]
2,5041237003,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,[380779709001001]
3,4487847009,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,[380150110012079]
4,2579588901,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,[380170006011035]
...,...,...,...,...,...
3217,2849968506,"3913 106th Ave SE, Sanborn, ND",46.848916,-98.228272,[380039680003112]
3218,3026678408,"1218 S Washington St, Grand Forks, ND",47.909497,-97.046115,[380350107002014]
3219,3175408401,"5458 134th Ave NW, Williston, ND",48.205512,-103.580819,[381059536002042]
3220,3860267409,"905 3RD AVE SW, WATFORD CITY, ND",47.796467,-103.295171,[380539624002036]


Remove brackets

In [14]:
fips_fill['FIPS_z'] = fips_fill['FIPS_z'].str[0]
fips_fill

,LoanNumber,full_add,Lat,Long,FIPS_z
0,5206477102,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,380350101002024
1,2976507101,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,380350117011097
2,5041237003,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,380779709001001
3,4487847009,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,380150110012079
4,2579588901,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,380170006011035
...,...,...,...,...,...
3217,2849968506,"3913 106th Ave SE, Sanborn, ND",46.848916,-98.228272,380039680003112
3218,3026678408,"1218 S Washington St, Grand Forks, ND",47.909497,-97.046115,380350107002014
3219,3175408401,"5458 134th Ave NW, Williston, ND",48.205512,-103.580819,381059536002042
3220,3860267409,"905 3RD AVE SW, WATFORD CITY, ND",47.796467,-103.295171,380539624002036


In [15]:
fips_fill.loc[fips_fill["FIPS_z"].isnull()] #check #24

,LoanNumber,full_add,Lat,Long,FIPS_z
688,4371027008,"202 6TH AVE, FLASHER, ND",40.726624,-74.002917,NaN
692,6225057007,"705 13th Ave S, WEST FARGO, ND",36.150132,-86.786585,NaN
930,2088118307,"10199 Lanesboro Rd, Scranton, ND",45.852712,-103.252859,NaN
1035,4443287005,"2355 Highway 10, MANDAN, ND",43.900656,-85.135021,NaN
1142,7274018602,"11801 Highway 2 and 52 W, Burlington, ND",47.597442,-120.649613,NaN
2469,7773717010,"369 Hwy 81 SE PO BOX 100, HILLSBORO, ND",33.437031,-97.762368,NaN
2492,8532788608,"369 HWY 81 SE, Hillsboro, ND",33.437031,-97.762368,NaN
2600,6843438007,"100 S 4TH ST, SUITE 104, FARGO, ND",46.853305,-96.773614,NaN
3205,2189357105,"103 9TH ST S, COOPERSTOWN, ND",46.873599,-96.766589,NaN
3214,9920748603,"320 Main St, Arthur, ND",33.743250,-118.103167,NaN


Parse for Merge

In [16]:
cols= ["LoanNumber", "FIPS_z"]
fips_fill_parsed = fips_fill[cols]
fips_fill_parsed

,LoanNumber,FIPS_z
0,5206477102,380350101002024
1,2976507101,380350117011097
2,5041237003,380779709001001
3,4487847009,380150110012079
4,2579588901,380170006011035
...,...,...
3217,2849968506,380039680003112
3218,3026678408,380350107002014
3219,3175408401,381059536002042
3220,3860267409,380539624002036


Read in base data

In [17]:
#if from geocode file
fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geocoded/geo_nd.csv")
#fips_check = fips_check.drop(["Unnamed: 0.1", "Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1", "Unnamed: 0.1.1.1.1.1.1"], axis =1) 
#fips_check

#if from FIPS file
#fips_check = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/ND_fips_scraped.csv") 
#fips_check = fips_check.drop([ "Unnamed: 0.1.1.1.1.1.1.1"], axis =1) 
fips_check.head() 


,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,546457,546457,546457,546457,546457,546457,546457,546457,546457,546457,...,999 Other,6233 Continuing Care Retirement Communities an...,6233.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,NaN
1,546458,546458,546458,546458,546458,546458,546458,546458,546458,546458,...,999 Other,"2373 Highway, Street, and Bridge Construction",2373.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,250 to 499,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,NaN
2,546459,546459,546459,546459,546459,546459,546459,546459,546459,546459,...,311 Food,3113 Sugar and Confectionery Product Manufactu...,3113.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,NaN
3,546460,546460,546460,546460,546460,546460,546460,546460,546460,546460,...,999 Other,2362 Nonresidential Building Construction,2362.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,250 to 499,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,NaN
4,546461,546461,546461,546461,546461,546461,546461,546461,546461,546461,...,999 Other,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,NaN


Merge with MainFrame

In [18]:
fips_merge = fips_check.merge(fips_fill_parsed, on="LoanNumber", how="left")
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y
0,546457,546457,546457,546457,546457,546457,546457,546457,546457,546457,...,6233 Continuing Care Retirement Communities an...,6233.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,NaN,380350101002024
1,546458,546458,546458,546458,546458,546458,546458,546458,546458,546458,...,"2373 Highway, Street, and Bridge Construction",2373.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,250 to 499,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,NaN,380350117011097
2,546459,546459,546459,546459,546459,546459,546459,546459,546459,546459,...,3113 Sugar and Confectionery Product Manufactu...,3113.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,NaN,380779709001001
3,546460,546460,546460,546460,546460,546460,546460,546460,546460,546460,...,2362 Nonresidential Building Construction,2362.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,250 to 499,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,NaN,380150110012079
4,546461,546461,546461,546461,546461,546461,546461,546461,546461,546461,...,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,NaN,380170006011035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,549674,549674,549674,549674,549674,549674,549674,549674,549674,549674,...,4841 General Freight Trucking,4841.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"3913 106th Ave SE, Sanborn, ND",46.848916,-98.228272,NaN,380039680003112
3218,549675,549675,549675,549675,549675,549675,549675,549675,549675,549675,...,7225 Restaurants and Other Eating Places,7225.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1218 S Washington St, Grand Forks, ND",47.909497,-97.046115,NaN,380350107002014
3219,549676,549676,549676,549676,549676,549676,549676,549676,549676,549676,...,2131 Support Activities for Mining,2131.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"5458 134th Ave NW, Williston, ND",48.205512,-103.580819,NaN,381059536002042
3220,549677,549677,549677,549677,549677,549677,549677,549677,549677,549677,...,7225 Restaurants and Other Eating Places,7225.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"905 3RD AVE SW, WATFORD CITY, ND",47.796467,-103.295171,NaN,380539624002036


<b>Fill FIPS Columns - if FIPS file

In [19]:
fips_merge["FIPS_z"] = fips_merge["FIPS_z_y"].fillna(fips_merge["FIPS_z_x"])
fips_merge

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z_x,FIPS_z_y,FIPS_z
0,546457,546457,546457,546457,546457,546457,546457,546457,546457,546457,...,6233.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,NaN,380350101002024,380350101002024
1,546458,546458,546458,546458,546458,546458,546458,546458,546458,546458,...,2373.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,250 to 499,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,NaN,380350117011097,380350117011097
2,546459,546459,546459,546459,546459,546459,546459,546459,546459,546459,...,3113.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,NaN,380779709001001,380779709001001
3,546460,546460,546460,546460,546460,546460,546460,546460,546460,546460,...,2362.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,250 to 499,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,NaN,380150110012079,380150110012079
4,546461,546461,546461,546461,546461,546461,546461,546461,546461,546461,...,6231.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,NaN,380170006011035,380170006011035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,549674,549674,549674,549674,549674,549674,549674,549674,549674,549674,...,4841.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"3913 106th Ave SE, Sanborn, ND",46.848916,-98.228272,NaN,380039680003112,380039680003112
3218,549675,549675,549675,549675,549675,549675,549675,549675,549675,549675,...,7225.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1218 S Washington St, Grand Forks, ND",47.909497,-97.046115,NaN,380350107002014,380350107002014
3219,549676,549676,549676,549676,549676,549676,549676,549676,549676,549676,...,2131.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"5458 134th Ave NW, Williston, ND",48.205512,-103.580819,NaN,381059536002042,381059536002042
3220,549677,549677,549677,549677,549677,549677,549677,549677,549677,549677,...,7225.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"905 3RD AVE SW, WATFORD CITY, ND",47.796467,-103.295171,NaN,380539624002036,380539624002036


Drop Extra FIPS Columns

In [20]:
fips_merge_drop = fips_merge.drop(["FIPS_z_x", "FIPS_z_y"], axis=1)
#fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1", "Unnamed: 0.1.1.1"], axis=1)
fips_merge_drop

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1,Unnamed: 0.1.1.1.1.1.1.1.1.1,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,546457,546457,546457,546457,546457,546457,546457,546457,546457,546457,...,999 Other,6233 Continuing Care Retirement Communities an...,6233.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,380350101002024
1,546458,546458,546458,546458,546458,546458,546458,546458,546458,546458,...,999 Other,"2373 Highway, Street, and Bridge Construction",2373.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,250 to 499,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,380350117011097
2,546459,546459,546459,546459,546459,546459,546459,546459,546459,546459,...,311 Food,3113 Sugar and Confectionery Product Manufactu...,3113.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,380779709001001
3,546460,546460,546460,546460,546460,546460,546460,546460,546460,546460,...,999 Other,2362 Nonresidential Building Construction,2362.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,250 to 499,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,380150110012079
4,546461,546461,546461,546461,546461,546461,546461,546461,546461,546461,...,999 Other,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,380170006011035
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,549674,549674,549674,549674,549674,549674,549674,549674,549674,549674,...,999 Other,4841 General Freight Trucking,4841.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"3913 106th Ave SE, Sanborn, ND",46.848916,-98.228272,380039680003112
3218,549675,549675,549675,549675,549675,549675,549675,549675,549675,549675,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1218 S Washington St, Grand Forks, ND",47.909497,-97.046115,380350107002014
3219,549676,549676,549676,549676,549676,549676,549676,549676,549676,549676,...,999 Other,2131 Support Activities for Mining,2131.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"5458 134th Ave NW, Williston, ND",48.205512,-103.580819,381059536002042
3220,549677,549677,549677,549677,549677,549677,549677,549677,549677,549677,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"905 3RD AVE SW, WATFORD CITY, ND",47.796467,-103.295171,380539624002036


In [23]:
fips_merge_drop = fips_merge_drop.drop(["Unnamed: 0.1.1", "Unnamed: 0.1.1.1","Unnamed: 0.1.1.1.1", "Unnamed: 0.1.1.1.1.1","Unnamed: 0.1.1.1.1.1.1"], axis=1)
fips_merge_drop.loc[fips_merge["FIPS_z"].isnull()] #verify #27 , notna #22799 / 22826

,Unnamed: 0,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,BorrowerZip,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
688,547145,4371027008,4/3/20,875,PPP,"MOR-GRAN-SOU ELECTRIC COOPERATIVE, INC",202 6TH AVE,FLASHER,ND,58535.0,...,2211 Electric Utilities,"2211 Electric Power Generation, Transmission a...",2211.0,NORTH DAKOTA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"202 6TH AVE, FLASHER, ND",40.726624,-74.002917,NaN
692,547149,6225057007,4/6/20,875,PPP,"BAKKEN CONTRACTING CO, LLC",705 13th Ave S,WEST FARGO,ND,58078.0,...,999 Other,2361 Residential Building Construction,2361.0,NORTH DAKOTA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"705 13th Ave S, WEST FARGO, ND",36.150132,-86.786585,NaN
930,547387,2088118307,1/20/21,875,PPS,FRONTIER HEAVY HAUL AND SUPPORT INC.,10199 Lanesboro Rd,Scranton,ND,58653.0,...,999 Other,4842 Specialized Freight Trucking,4842.0,NORTH DAKOTA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"10199 Lanesboro Rd, Scranton, ND",45.852712,-103.252859,NaN
1035,547492,4443287005,4/3/20,875,PPP,"ROUGHRIDER TRANSPORTATION, INC.",2355 Highway 10,MANDAN,ND,58554.0,...,999 Other,4841 General Freight Trucking,4841.0,NORTH DAKOTA DISTRICT OFFICE,"b $350,000-1 million",20 to 49,"2355 Highway 10, MANDAN, ND",43.900656,-85.135021,NaN
1142,547599,7274018602,3/23/21,875,PPP,TRAVIS IRWIN,11801 Highway 2 and 52 W,Burlington,ND,58722.0,...,999 Other,2131 Support Activities for Mining,2131.0,NORTH DAKOTA DISTRICT OFFICE,"b $350,000-1 million",10 to 19,"11801 Highway 2 and 52 W, Burlington, ND",47.597442,-120.649613,NaN
2469,548926,7773717010,4/8/20,875,PPP,"ALTON GRAIN TERMINAL, LLC",369 Hwy 81 SE PO BOX 100,HILLSBORO,ND,58045.0,...,999 Other,4931 Warehousing and Storage,4931.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"369 Hwy 81 SE PO BOX 100, HILLSBORO, ND",33.437031,-97.762368,NaN
2492,548949,8532788608,3/25/21,875,PPS,ALTON GRAIN TERMINAL LLC,369 HWY 81 SE,Hillsboro,ND,58045.0,...,999 Other,4931 Warehousing and Storage,4931.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"369 HWY 81 SE, Hillsboro, ND",33.437031,-97.762368,NaN
2600,549057,6843438007,6/30/20,875,PPP,PROSPERITY PHARMACY SOLUTIONS INC.,"100 S 4TH ST, SUITE 104",FARGO,ND,58103.0,...,999 Other,4461 Health and Personal Care Stores,4461.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"100 S 4TH ST, SUITE 104, FARGO, ND",46.853305,-96.773614,NaN
3205,549662,2189357105,4/10/20,875,PPP,"COOPERSTOWN IMPLEMENT COMPANY, INC",103 9TH ST S,COOPERSTOWN,ND,58425.0,...,999 Other,"4238 Machinery, Equipment, and Supplies Mercha...",4238.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",10 to 19,"103 9TH ST S, COOPERSTOWN, ND",46.873599,-96.766589,NaN
3214,549671,9920748603,3/26/21,875,PPS,AGRI VALLEY INSURANCE INC.,320 Main St,Arthur,ND,58006.0,...,999 Other,"5242 Agencies, Brokerages, and Other Insurance...",5242.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"320 Main St, Arthur, ND",33.743250,-118.103167,NaN


Convert to File

In [24]:
fips_ND = fips_merge_drop


In [25]:
fips_ND.to_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/ND_fips_scraped.csv") 

Review & Compare

In [26]:
fips_ND = pd.read_csv("../../turbo-octo-goggles/data/state_data/geo/geo_fips/ND_fips_scraped.csv") 
fips_ND

,Unnamed: 0,Unnamed: 0.1,LoanNumber,DateApproved,SBAOfficeCode,ProcessingMethod,BorrowerName,BorrowerAddress,BorrowerCity,BorrowerState,...,TRIIndustrySector,IndustrySubsector,4DigitNAICS,SBAOfficeLabel,Loan_Range,job_Range,full_add,Lat,Long,FIPS_z
0,0,546457,5206477102,4/13/20,875,PPP,EDGEWOOD OPCO LLC,322 Demers Ave Suite 500,GRAND FORKS,ND,...,999 Other,6233 Continuing Care Retirement Communities an...,6233.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"322 Demers Ave Suite 500, GRAND FORKS, ND",47.924918,-97.031593,3.803501e+14
1,1,546458,2976507101,4/11/20,875,PPP,STRATA CORPORATION,1600 N 48TH ST,GRAND FORKS,ND,...,999 Other,"2373 Highway, Street, and Bridge Construction",2373.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,250 to 499,"1600 N 48TH ST, GRAND FORKS, ND",47.936733,-97.098896,3.803501e+14
2,2,546459,5041237003,4/4/20,875,PPP,MINN-DAK FARMERS COOPERATIVE,7525 RED RIVER RD,WAHPETON,ND,...,311 Food,3113 Sugar and Confectionery Product Manufactu...,3113.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"7525 RED RIVER RD, WAHPETON, ND",46.324042,-96.615776,3.807797e+14
3,3,546460,4487847009,4/3/20,875,PPP,"MILLER INSULATION CO, INC.",3520 E. Century Avenue,BISMARCK,ND,...,999 Other,2362 Nonresidential Building Construction,2362.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,250 to 499,"3520 E. Century Avenue, BISMARCK, ND",46.838560,-100.735451,3.801501e+14
4,4,546461,2579588901,4/27/21,875,PPP,SISTERS OF MARY OF THE PRESENTATION LONG TERM ...,1202 Page Dr S,Fargo,ND,...,999 Other,6231 Nursing Care Facilities (Skilled Nursing ...,6231.0,NORTH DAKOTA DISTRICT OFFICE,e $5-10 million,500 or more,"1202 Page Dr S, Fargo, ND",46.863174,-96.827143,3.801700e+14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3217,3217,549674,2849968506,2/22/21,875,PPS,G & R TRUCKING LLC,3913 106th Ave SE,Sanborn,ND,...,999 Other,4841 General Freight Trucking,4841.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"3913 106th Ave SE, Sanborn, ND",46.848916,-98.228272,3.800397e+14
3218,3218,549675,3026678408,2/4/21,875,PPS,H.J. VENTURE CORP.,1218 S Washington St,Grand Forks,ND,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"1218 S Washington St, Grand Forks, ND",47.909497,-97.046115,3.803501e+14
3219,3219,549676,3175408401,2/4/21,875,PPS,NORTHWEST WATER TRANSFER LLC,5458 134th Ave NW,Williston,ND,...,999 Other,2131 Support Activities for Mining,2131.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",5 to 9,"5458 134th Ave NW, Williston, ND",48.205512,-103.580819,3.810595e+14
3220,3220,549677,3860267409,5/8/20,875,PPP,OUTSIDER'S BAR AND GRILL LLC,905 3RD AVE SW,WATFORD CITY,ND,...,999 Other,7225 Restaurants and Other Eating Places,7225.0,NORTH DAKOTA DISTRICT OFFICE,"a $150,000-350,000",20 to 49,"905 3RD AVE SW, WATFORD CITY, ND",47.796467,-103.295171,3.805396e+14
